# Введение в статистику и тесты SciPy

Короткое теоретическое введение и компактный обзор статистических тестов


## 1. Расширенная теория

### 1.1 Описательная статистика
- **Среднее** и **медиана** описывают центр распределения.
- **Дисперсия** и **стандартное отклонение** описывают разброс.
- **Квартили** и **IQR** помогают видеть форму и устойчивы к выбросам.

### 1.2 Выборка и генеральная совокупность
- Мы наблюдаем **выборку**, но хотим сделать выводы о **популяции**.
- Случайность отбора и достаточный размер выборки определяют надежность выводов.
- Центральная предельная теорема: при большом n распределение среднего стремится к нормальному, даже если данные не нормальные.

### 1.3 Параметры и оценки
- **Параметры** популяции неизвестны (например, истинное среднее).
- **Оценки** вычисляются по выборке (выборочное среднее, выборочная дисперсия).
- Оценки имеют **смещение** и **дисперсию**; важен баланс.

### 1.4 Доверительные интервалы
- Доверительный интервал показывает диапазон правдоподобных значений параметра.
- 95% интервал не означает 95% вероятность для параметра, а означает стабильность метода в повторяемых экспериментах.

### 1.5 Проверка гипотез
- **H0**: эффекта нет, **H1**: эффект есть.
- **p-value**: вероятность наблюдать данные не менее экстремальные при истинной H0.
- Уровень значимости **alpha** (обычно 0.05) — порог для отклонения H0.
- **Односторонние** и **двусторонние** тесты задают направление альтернативы.

### 1.6 Ошибки и мощность
- **Ошибка I рода**: ложноположительный вывод (отклоняем H0, когда она верна).
- **Ошибка II рода**: ложноотрицательный вывод (не отклоняем H0, когда она неверна).
- **Мощность теста** = 1 − вероятность ошибки II рода; растет с размером выборки и эффектом.

### 1.7 Размер эффекта
- p-value говорит о статистической значимости, но не о практической важности.
- Размер эффекта (например, разница средних, корреляция) важен для интерпретации.

### 1.8 Предпосылки тестов
- Параметрические тесты обычно предполагают нормальность и гомогенность дисперсий.
- При нарушениях предпосылок используйте непараметрические аналоги или трансформации.
- Диагностика: графики, тесты нормальности, проверка выбросов.

### 1.9 Множественные проверки
- При множественных тестах растет доля ложных находок.
- Коррекция p-value (например, Benjamini–Hochberg) контролирует FDR.

### 1.10 Практические советы
- Начинайте с визуализации и проверки предпосылок.
- Выбирайте тест по дизайну эксперимента и типу данных.
- Интерпретируйте вместе: p-value + размер эффекта + доверительный интервал.


### 1.11 Иллюстрации: распределение, выбросы, нормальность
- Гистограмма показывает форму распределения.
- Boxplot помогает быстро увидеть медиану, IQR и выбросы.
- QQ-plot сравнивает данные с нормальным распределением.


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Данные для иллюстраций
normal_data = np.random.normal(loc=0, scale=1, size=200)
skewed_data = np.random.exponential(scale=1.0, size=200)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Гистограммы
axes[0].hist(normal_data, bins=20, alpha=0.7, label='Normal')
axes[0].hist(skewed_data, bins=20, alpha=0.7, label='Skewed')
axes[0].set_title('Histogram')
axes[0].legend()

# Boxplot
axes[1].boxplot([normal_data, skewed_data], labels=['Normal', 'Skewed'])
axes[1].set_title('Boxplot')

# QQ-plot для нормальных данных
stats.probplot(normal_data, dist='norm', plot=axes[2])
axes[2].set_title('QQ-plot (Normal)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import false_discovery_control

np.random.seed(42)


## 2. Статистические тесты (scipy.stats)

Ниже — все тесты, которые встречаются в примере по SciPy.


### 2.1 t-test: сравнение средних
- **Независимый t-test**: сравнение средних двух независимых групп.
- **Парный t-test**: сравнение средних до/после для тех же объектов.

Предпосылки: приблизительная нормальность и равенство дисперсий (для независимого теста).


In [ ]:
# Независимый t-test
control = np.random.normal(loc=10, scale=2, size=30)
treated = np.random.normal(loc=11, scale=2, size=30)
t_stat, p_value = stats.ttest_ind(control, treated)
print(f"t-test (ind): t={t_stat:.3f}, p={p_value:.4f}")

# Парный t-test
before = np.random.normal(loc=50, scale=5, size=20)
after = before + np.random.normal(loc=-2, scale=3, size=20)
t_stat_paired, p_value_paired = stats.ttest_rel(before, after)
print(f"t-test (paired): t={t_stat_paired:.3f}, p={p_value_paired:.4f}")


### 2.2 Mann–Whitney U test
Непараметрический тест для сравнения двух независимых групп по ранговым значениям.


In [ ]:
group1 = np.random.normal(loc=100, scale=15, size=25)
group2 = np.random.normal(loc=110, scale=15, size=25)
u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')
print(f"Mann-Whitney U: U={u_stat:.3f}, p={p_value_mw:.4f}")


### 2.3 ANOVA (однофакторный тест)
Сравнение средних более чем двух независимых групп.


In [ ]:
dose_0 = np.random.normal(loc=5, scale=1, size=20)
dose_1 = np.random.normal(loc=6, scale=1, size=20)
dose_2 = np.random.normal(loc=7, scale=1, size=20)
f_stat, p_value_anova = stats.f_oneway(dose_0, dose_1, dose_2)
print(f"ANOVA: F={f_stat:.3f}, p={p_value_anova:.4f}")


### 2.4 Chi-square test (хи-квадрат)
Проверяет независимость категориальных признаков в таблице сопряженности.


In [ ]:
contingency_table = np.array([[10, 20, 30], [6,  9, 17]])
chi2, p_value_chi, dof, expected = stats.chi2_contingency(contingency_table)
print(f"Chi-square: chi2={chi2:.3f}, p={p_value_chi:.4f}, dof={dof}")


### 2.5 Корреляция
- **Pearson**: линейная корреляция, чувствителен к нормальности и выбросам.
- **Spearman**: ранговая корреляция, устойчивее к выбросам и нелинейности.


In [ ]:
gene_a = np.random.normal(loc=100, scale=20, size=30)
gene_b = gene_a * 0.8 + np.random.normal(loc=0, scale=10, size=30)

r_pearson, p_pearson = stats.pearsonr(gene_a, gene_b)
r_spearman, p_spearman = stats.spearmanr(gene_a, gene_b)
print(f"Pearson: r={r_pearson:.3f}, p={p_pearson:.4f}")
print(f"Spearman: r={r_spearman:.3f}, p={p_spearman:.4f}")


### 2.6 Проверка нормальности (Shapiro–Wilk)
Проверяет гипотезу о нормальном распределении.


In [ ]:
normal_data = np.random.normal(loc=0, scale=1, size=50)
non_normal_data = np.random.exponential(scale=1.0, size=50)

stat_norm, p_norm = stats.shapiro(normal_data)
stat_non, p_non = stats.shapiro(non_normal_data)
print(f"Shapiro (normal): W={stat_norm:.3f}, p={p_norm:.4f}")
print(f"Shapiro (non-normal): W={stat_non:.3f}, p={p_non:.4f}")


### 2.7 Множественное тестирование (FDR)
Коррекция p-value при множественных проверках, чтобы контролировать FDR.


In [ ]:
p_values = np.array([0.001, 0.01, 0.02, 0.04, 0.06, 0.2])
adj_p = false_discovery_control(p_values, method='bh')
for p, adj in zip(p_values, adj_p):
    print(f"p={p:.3f} -> adj_p={adj:.3f}")
